# Домашнее задание к теме Базовые понятия статистики

## Обязательная часть

Будем осуществлять работу с непростым [набором данных](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.csv) о состоянии здоровья лошадей, испытывающих кишечные колики.

### Задание 1. Базовое изучение

Изучить представленный набор данных на основе [описания его столбцов](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.names) и выбрать 8 столбцов для дальнейшего изучения (среди них должны быть как числовые, так и категориальные). Провести расчет базовых метрик для них, кратко описать результаты.

### Задание 2. Работа с выбросами

В выбранных числовых столбцах найти выбросы, выдвинуть гипотезы об их причинах и проинтерпретировать результаты. Принять и обосновать решение о дальнейшей работе с ними.

### Задание 3. Работа с пропусками

Рассчитать количество выбросов для всех выбранных столбцов. Принять и обосновать решение о методе работы с пропусками по каждому столбцу, сформировать датафрейм, в котором пропуски будут отсутствовать.

## Дополнительная часть (необязательная)

Выполнить задания 1-3 для всего набора данных.

7. Информация об атрибутах (ключевые параметры):

  4:  rectal temperature — Ректальная температура  (числовой)  
          - линейная  
          - в градусах Цельсия  
          - повышенная температура возможна при инфекции  
          - может снижаться при поздней стадии шока  
          - нормальная температура — 37.8  
          - этот параметр обычно меняется по мере развития проблемы  
               например: может начинаться нормальной, затем повышаться из-за  
                   поражения, снова становиться нормальной, когда лошадь впадает в шок  

  9:  mucous membranes — Слизистые оболочки  (категориальный)  
          - субъективная оценка окраски  
          - возможные значения:  
               1 = нормально-розовые  
               2 = ярко-розовые  
               3 = бледно-розовые  
               4 = бледно-цианотичные  
               5 = ярко-красные / воспалённые  
               6 = тёмно-цианотичные  
          - 1 и 2 — нормальное или слегка усиленное кровообращение  
          - 3 — возможно на ранней стадии шока  
          - 4 и 6 — признак серьёзного нарушения циркуляции  
          - 5 — указывает на септицемию  

 11: pain — Боль  (категориальный, субъективный)  
          - субъективная оценка уровня боли  
          - возможные значения:  
               1 = бодрая, без боли  
               2 = подавленная  
               3 = перемежающаяся лёгкая боль  
               4 = перемежающаяся сильная боль  
               5 = постоянная сильная боль  
          - не следует рассматривать как упорядоченную переменную  
          - чем сильнее боль, тем выше вероятность необходимости операции  
          - обезболивание может частично скрыть уровень боли  

 12: peristalsis — Перистальтика  (категориальный)  
          - показатель активности кишечника; при вздутии или токсичности снижается  
          - возможные значения:  
               1 = гипермоторика  
               2 = нормальная  
               3 = гипомоторика  
               4 = отсутствует  

 13: abdominal distension — Вздутие живота  (категориальный)  
          - ВАЖНЫЙ параметр  
          - возможные значения:  
               1 = нет  
               2 = лёгкое  
               3 = умеренное  
               4 = сильное  
          - сильное вздутие часто требует хирургического вмешательства  

 15: nasogastric reflux — Назогастральный рефлюкс  (категориальный / количественный, объём)  
          - возможные значения:  
               1 = нет  
               2 = > 1 литра  
               3 = < 1 литра  
          - большой объём рефлюкса — признак обструкции кишечника  

 19: packed cell volume — Гематокрит  (числовой)  
          - линейный  
          - процент эритроцитов в крови  
          - норма: 30–50  
          - уровень повышается при обезвоживании или нарушении кровообращения

In [ ]:
import pandas as pd

attributes_ru = [
    'Операция',
    'Возраст',
    'Номер госпитализации',
    'Ректальная температура',
    'Пульс',
    'Частота дыхания',
    'Температура конечностей',
    'Периферический пульс',
    'Слизистые оболочки',
    'Время капиллярного наполнения',
    'Боль',
    'Перистальтика',
    'Вздутие живота',
    'Назогастральная трубка',
    'Назогастральный рефлюкс',
    'pH назогастрального рефлюкса',
    'Ректальный осмотр — кал',
    'Живот',
    'Гематокрит',
    'Общий белок',
    'Внешний вид жидкости при абдоминоцентезе',
    'Белок при абдоминоцентезе',
    'Исход',
    'Хирургическое поражение',
    'Тип поражения (25)',
    'Тип поражения (26)',
    'Тип поражения (27)',
    'Наличие патологоанатомических данных'
]

horses = pd.read_csv("horse_data.csv")
horses.columns = attributes_ru

horses_important = horses[[
    'Ректальная температура',              # отражает инфекцию, шок
    'Пульс',                               # повышается при боли и шоке
    'Слизистые оболочки',                  # указывает на степень шока и перфузию тканей
    'Боль',                                # основной симптом тяжести состояния
    'Перистальтика',                       # снижение указывает на кишечную непроходимость
    'Вздутие живота',                      # важнейший показатель, часто требует операции
    'Назогастральный рефлюкс',             # большой объём — признак обструкции
    'Гематокрит'                           # повышен при обезвоживании и шоке
]]

# display(horses_important.head())
display(rectal_temperature.head())

,Ректальная температура
0,39.2
1,38.30
2,39.10
3,37.30
4,?
